<a href="https://colab.research.google.com/github/heyisula/fineweb-edu-llm-training/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Setup

In [8]:
import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

In [9]:
# Install dependencies (Colab environments usually need these)
!pip install datasets transformers tqdm faiss-cpu sentence-transformers adafactor

  Using cached faiss_cpu-1.13.2-cp310-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (7.6 kB)
ERROR: Could not find a version that satisfies the requirement adafactor (from versions: none)
ERROR: No matching distribution found for adafactor


In [10]:
# Mount Google Drive to save the model permanently
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Fine-Tune GPT-2 Large on FineWeb-Edu

This notebook fine-tunes the **GPT-2 Large (774M params)** pre-trained model on high-quality educational content from FineWeb-Edu.

**Hardware**: Optimized for **T4 GPU (15GB VRAM)** on Google Colab.

In [11]:
import torch
from datasets import load_dataset, Dataset
from transformers import (
    GPT2LMHeadModel,
    GPT2TokenizerFast,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
import os
from tqdm import tqdm

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

Using device: cuda
CUDA available: True
GPU: Tesla T4
VRAM: 14.6 GB


## 1. Load Pre-trained GPT-2 Large Model & Tokenizer

We upgrade to **GPT-2 Large (774M params)** for much better reasoning and knowledge capacity.

In [13]:
MODEL_NAME = "gpt2-large"

print(f"Loading pre-trained model: {MODEL_NAME}...")
tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

model.gradient_checkpointing_enable()
# Enabling gradient checkpointing to fit 774M params in 15GB VRAM

num_params = sum(p.numel() for p in model.parameters())
print(f"Model loaded: {num_params / 1e6:.1f}M parameters")
print(f"Max context length: {model.config.n_positions}")

Loading pre-trained model: gpt2-large...


Loading weights:   0%|          | 0/436 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2-large
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...35}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded: 774.0M parameters
Max context length: 1024


## 2. Load & Materialize FineWeb-Edu Dataset

Using **700,000 samples** for the larger model's training.

In [14]:
NUM_SAMPLES = 700_000

print(f"Loading {NUM_SAMPLES:,} samples from FineWeb-Edu (streaming)...")
dataset = load_dataset(
    "HuggingFaceFW/fineweb-edu",
    split="train",
    streaming=True
)

# Materializing
subset_iter = dataset.take(NUM_SAMPLES)
data_list = [row for row in tqdm(subset_iter, total=NUM_SAMPLES, desc="Materializing dataset")]
print(f"Total samples materialized: {len(data_list):,}")
print(f"Sample keys: {data_list[0].keys()}")

Loading 700,000 samples from FineWeb-Edu (streaming)...


Resolving data files:   0%|          | 0/2410 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2410 [00:00<?, ?it/s]

Materializing dataset: 100%|██████████| 700000/700000 [15:35<00:00, 748.19it/s]

Total samples materialized: 700,000
Sample keys: dict_keys(['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score'])


## 3. Tokenize the Dataset

In [1]:
MAX_LENGTH = 1024

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )

print("Converting to HuggingFace Dataset...")
hf_dataset = Dataset.from_list(data_list)

print("Tokenizing (this may take 15-30 mins for 1M samples)...")
tokenized_dataset = hf_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=1000,
    remove_columns=hf_dataset.column_names,
    desc="Tokenizing"
)

print(f"Tokenized dataset: {len(tokenized_dataset):,} samples")
print(f"Token sequence length: {MAX_LENGTH}")

Converting to HuggingFace Dataset...


NameError: name 'Dataset' is not defined

## 4. Configure Training

Optimized for **T4 15GB VRAM**:
- **Batch size 2** + accumulation 8 = effective batch 16
- **Adafactor** + **FP16** + **Gradient Checkpointing** enabled

In [ ]:
output_dir = "out/models/gpt2_large_finetuned"
os.makedirs(output_dir, exist_ok=True)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,   # Increased for T4
    gradient_accumulation_steps=8,   # Effective batch 16
    learning_rate=2e-5,
    num_train_epochs=1,
    fp16=True,
    gradient_checkpointing=True,
    optim="adafactor",
    logging_steps=100,
    save_steps=2000,
    save_total_limit=2,
    report_to="none",
    dataloader_num_workers=4
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

print(f"Training config:")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Total steps: {len(tokenized_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")
print(f"  Optimizer: {training_args.optim}")
print(f"  FP16: {training_args.fp16}")
print(f"  Gradient checkpointing: {training_args.gradient_checkpointing}")

## 5. Train

In [ ]:
print("Starting fine-tuning on T4 cloud...")
trainer.train()

In [ ]:
print("Saving model and tokenizer...")
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to: {output_dir}")
print("Fine-tuning complete!")

## 6. Build RAG Index

In [ ]:
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

RAG_SAMPLES = 100_000
RAG_DIR = "out/rag_index"
os.makedirs(RAG_DIR, exist_ok=True)

passages = []
for row in tqdm(data_list[:RAG_SAMPLES], desc="Extracting passages"):
    text = row["text"].strip()
    for i in range(0, len(text), 500):
        chunk = text[i:i + 500].strip()
        if len(chunk) > 50: passages.append(chunk)

print(f"Total passages: {len(passages):,}")

print("Loading sentence-transformer model...")
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

print("Encoding passages (this may take a few minutes)...")
embeddings = embedder.encode(passages, show_progress_bar=True, batch_size=256, convert_to_numpy=True)

print("Building FAISS index...")
index = faiss.IndexFlatIP(embeddings.shape[1])
faiss.normalize_L2(embeddings)
index.add(embeddings)

faiss.write_index(index, os.path.join(RAG_DIR, "faiss_index.bin"))
np.save(os.path.join(RAG_DIR, "passages.npy"), np.array(passages, dtype=object))
print("RAG Index Built!")
print(f"RAG index saved to {RAG_DIR}/")
print(f"  Index: {index.ntotal:,} vectors, {dimension}D")
print(f"  Passages: {len(passages):,}")
print("Done!")

## 7. Save to Google Drive
Run this to persist your model so you can download it locally.

In [ ]:
import shutil
drive_path = "/content/drive/MyDrive/fineweb_edu_gpt2_large"
os.makedirs(drive_path, exist_ok=True)

print("Copying model to Google Drive...")
shutil.copytree(output_dir, os.path.join(drive_path, "model"), dirs_exist_ok=True)
shutil.copytree(RAG_DIR, os.path.join(drive_path, "rag_index"), dirs_exist_ok=True)
print(f"All files saved to: {drive_path}")